# Exercise 2: Vegetation Index Analysis
In this exercise, you will analyze vegetation changes within one growing season in a selected agricultural region (Kujawy, Poland).

## Task 1: Define AOI
Create an Area of Interest (AOI) using the coordinates provided below.

In [8]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-jsumara')


In [12]:
# TODO: Define AOI
aoi = ee.Geometry.Rectangle([21.85, 50.05, 21.74, 50.00])

## Task 2: Retrieve Sentinel-2 Imagery
Retrieve two Sentinel-2 satellite images from the following dates: 2023-04-01 and 2023-07-01.

In [31]:
def get_image(date, aoi):
    date = ee.Date(date)
    start = date.advance(-15, 'day')
    end = date.advance(15, 'day')

    collection = (
        ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))
        .sort('CLOUDY_PIXEL_PERCENTAGE')
    )
    return collection.first()

image_april = get_image('2023-04-01', aoi)
image_july = get_image('2023-07-01', aoi)

common_area = image_april.geometry().intersection(image_july.geometry(), ee.ErrorMargin(1))
image_april = image_april.clip(common_area); image_july = image_july.clip(common_area)


Map = geemap.Map(center=[50, 22], zoom=8)

vis_params = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.3}

Map.addLayer(image_april, vis_params, 'Sentinel-2 April 2023')
Map.addLayer(image_july, vis_params, 'Sentinel-2 July 2023')
Map



Map(center=[50, 22], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

## Task 3: Calculate NDVI
Calculate the Normalized Difference Vegetation Index (NDVI) for both images.

In [32]:
# TODO: Calculate NDVI for both images (Choose the appropriate S-2 bands).
ndvi_april = image_april.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_july = image_july.normalizedDifference(['B8', 'B4']).rename('NDVI')

Map.addLayer(ndvi_april, {'min': 0, 'max': 1, 'palette': ['white', 'darkgreen']}, 'NDVI April 2023')
Map.addLayer(ndvi_july, {'min': 0, 'max': 1, 'palette': ['white', 'darkgreen']}, 'NDVI July 2023')

Map

Map(bottom=22526.0, center=[50, 22], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

## Task 4: Compare NDVI
Display both NDVI indices on the map and calculate and visualize their difference.

In [35]:
# TODO: Compare NDVI, visualize the differences, and analyze vegetation changes.
ndvi_diff = ndvi_july.subtract(ndvi_april).rename('NDVI_Diff')

Map.addLayer(ndvi_diff, {
    'min': -0.5, 'max': 0.5,
    'palette': ['red', 'white', 'green']
}, 'NDVI Difference')

Map


Map(bottom=713289.0, center=[49.80641889166585, 21.660833358764652], controls=(WidgetControl(options=['positio…

## Task 5: Interpret the Results
Briefly describe the meaning of the observed NDVI changes within the analyzed period.

Color red means negative change (lower NDVI -- clouds). white means no change (buildings, roads, water bodies etc.) 
and green means positive change (higher NDVI = vegetation growth).
Big positive change is seen mainly in forests (more grean leaves in July than in April),

## Task 6: False Color Composite for Vegetation Condition Assessment
Add false-color composites (NIR, RED, GREEN) for both dates to the map for visual assessment of vegetation condition.

In [ ]:
# TODO: Create a false-color composite (NIR, RED, GREEN) and add it to the map
vegetation_viz = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 4000}
# Add Sentinel-2 images for both dates to the map using this composite.
Map.addLayer(image_april, vegetation_viz, 'False Color April 2023')
Map.addLayer(image_july, vegetation_viz, 'False Color July 2023')

Map

Map(bottom=44661.0, center=[50.161064818858684, 22.35031127929688], controls=(WidgetControl(options=['position…